In [24]:
import pandas as pd
import smtplib
import email.message

# Operando as bases de dados

In [15]:
tabela_vendas = pd.read_excel('Vendas.xlsx')

tabela_vendas.head(4)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,Iguatemi Esplanada,Sapato Estampa,1,358,358
1,1,2019-01-01,Iguatemi Esplanada,Camiseta,2,180,360
2,1,2019-01-01,Iguatemi Esplanada,Sapato Xadrez,1,368,368
3,2,2019-01-02,Norte Shopping,Relógio,3,200,600


In [16]:
tabela_vendas[['ID Loja', 'Valor Final']]

,ID Loja,Valor Final
0,Iguatemi Esplanada,358
1,Iguatemi Esplanada,360
2,Iguatemi Esplanada,368
3,Norte Shopping,600
4,Norte Shopping,71
...,...,...
100994,Center Shopping Uberlândia,204
100995,Center Shopping Uberlândia,1080
100996,Center Shopping Uberlândia,87
100997,Ribeirão Shopping,108


Faturamento por loja

In [40]:
def agrupador(tabela_vendas, referencia):
    # Iremos agrupar, por loja, e passar o que se precisa fazer com as outras colunas
    df = tabela_vendas[['ID Loja', referencia]].groupby('ID Loja').sum()

    # Criando um ranking com base na "referencia"
    desempenho = df.rank(ascending = False)

    # Renomeando a coluna com os rankings
    desempenho.columns = ['Ranking']

    # Fazendo uma pequena modificação na exibição das posiçoes, para que fique
    # mais intuitivo o que se deseja passar
    desempenho['Ranking'] = desempenho['Ranking'].apply(lambda x: str(int(x)) + 'º')

    # concatenando o agrupamento com os rankings
    df = pd.concat([df, desempenho], axis = 1)

    return df

In [42]:
faturamento = agrupador(tabela_vendas, 'Valor Final')
quantidade = agrupador(tabela_vendas, 'Quantidade')

display(faturamento)
display(quantidade)

,Valor Final,Ranking
ID Loja,,
Bourbon Shopping SP,1726110,3º
Center Shopping Uberlândia,1668921,14º
Iguatemi Campinas,1762419,1º
Iguatemi Esplanada,1699681,9º
Norte Shopping,1711968,7º
Novo Shopping Ribeirão Preto,1678225,12º
Palladium Shopping Curitiba,1721120,6º
Parque Dom Pedro Shopping,1631630,22º
Passei das Águas Shopping,1649014,17º


,Quantidade,Ranking
ID Loja,,
Bourbon Shopping SP,8863,8º
Center Shopping Uberlândia,8627,19º
Iguatemi Campinas,8935,6º
Iguatemi Esplanada,8580,22º
Norte Shopping,9014,3º
Novo Shopping Ribeirão Preto,8751,11º
Palladium Shopping Curitiba,9091,2º
Parque Dom Pedro Shopping,8388,24º
Passei das Águas Shopping,8618,20º


Faturamento por produto

In [30]:
ticket_medio = (faturamento['Valor Final'] / quantidade['Quantidade']).to_frame()
ticket_medio.columns = ['Ticket Médio']
ticket_medio = ticket_medio.rename(columns = {0: 'Ticket Médio'})
ticket_medio

,Ticket Médio
ID Loja,
Bourbon Shopping SP,194.754598
Center Shopping Uberlândia,193.453228
Iguatemi Campinas,197.248909
Iguatemi Esplanada,198.098019
Norte Shopping,189.923231
Novo Shopping Ribeirão Preto,191.775226
Palladium Shopping Curitiba,189.321307
Parque Dom Pedro Shopping,194.519552
Passei das Águas Shopping,191.345324


# Enviando email automaticamente

In [43]:

corpo_email = f''' 
Prezados, 

Segue o relatório de vendas por cada loja.
<p>Faturamento:<p>
{faturamento.to_html(formatters = {'Valor Final': 'R${:,.2f}'.format})}

<p>Quantidade Vendida:<p>
{quantidade.to_html(formatters = {'Quantidade': 'R${:,.2f}'.format})}

<p>Ticket Médio dos Produtos em cada Loja:<p>
{ticket_medio.to_html(formatters = {'Ticket Médio': 'R${:,.2f}'.format})}

<p>att..<p>
<p>Kaio<p>

<p>Em caso de dúvidas, o código estará no github kaioemmartins@gmail.com<p>
'''

msg = email.message.Message()
msg['Subject'] = 'Automação no minicurso da Hashtag'
msg['From'] = 'kaioemmartins@gmail.com'
msg['To'] = 'kaio.martins@itec.ufpa.br'

# Password deletado após execução do código, mas fica de exemplo
password = 'zhoa xfdw autm pcek'
msg.add_header('Content-Type', 'text/html')
msg.set_payload(corpo_email)

s = smtplib.SMTP('smtp.gmail.com: 587')
s.starttls()

s.login(msg['From'], password)
s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))

print('email enviado')

email enviado
